In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
path = '/content/drive/My Drive/Final Project'
sys.path.append(path)

%cd $path

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/18oPzXa_o2Y_k8Bcbz3FNVsqteJ0Pt29Y/Final Project


In [ ]:
# HuggingFace transformers library (contains all the models we'll be using)
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 43.1 MB/s 
     |████████████████████████████████| 182 kB 68.7 MB/s 
     |████████████████████████████████| 7.6 MB 54.2 MB/s 


In [ ]:
from transformers import TFAutoModelForMaskedLM
from transformers import AutoTokenizer
import numpy as np
import tensorflow as tf

In [ ]:
model_checkpoint = "distilbert-base-uncased"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [ ]:
model.summary()

Model: "tf_distil_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 vocab_transform (Dense)     multiple                  590592    
                                                                 
 vocab_layer_norm (LayerNorm  multiple                 1536      
 alization)                                                      
                                                                 
 vocab_projector (TFDistilBe  multiple                 23866170  
 rtLMHead)                                                       
                                                                 
Total params: 66,985,530
Trainable params: 66,985,530
Non-trainable params: 0
__________________________

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def singleToken(sentence):
  inputs = tokenizer(sentence, return_tensors="np")
  token_logits = model(**inputs).logits
  mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0,1]
  mask_token_logits = token_logits[0, mask_token_index, :]
  top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()
  for token in top_5_tokens:
    print(f">>> {negative_example.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

In [ ]:
negative_example = "The crypto market is going to [MASK] today. It lost 10 points on the exchange."

In [ ]:
singleToken(negative_example)

>>> The crypto market is going to collapse today. It lost 10 points on the exchange.
>>> The crypto market is going to fail today. It lost 10 points on the exchange.
>>> The crypto market is going to crash today. It lost 10 points on the exchange.
>>> The crypto market is going to suffer today. It lost 10 points on the exchange.
>>> The crypto market is going to dominate today. It lost 10 points on the exchange.


In [ ]:
positive_example = "The crypto market is going to [MASK] today. It rose 10 points on the exchange."

In [ ]:
singleToken(negative_example)

>>> The crypto market is going to collapse today. It lost 10 points on the exchange.
>>> The crypto market is going to fail today. It lost 10 points on the exchange.
>>> The crypto market is going to crash today. It lost 10 points on the exchange.
>>> The crypto market is going to suffer today. It lost 10 points on the exchange.
>>> The crypto market is going to dominate today. It lost 10 points on the exchange.


In [ ]:
# STRUCTURE = "The [MASK] market is going to [MASK] today."
# TOPICS = ["Finance", "Bad"]
# COMBINED = STRUCTURE + " " + " ".join(TOPICS)
# OPTIONS_PER_MASK = 2 # Note this grows exponentially with number of masks (n). 3^n

STRUCTURE = "Today I went to [MASK] but I checked my phone and saw that the [MASK] market [MASK]."
TOPICS = ["Finance", "Bad"]
COMBINED = STRUCTURE + " " + " ".join(TOPICS)
OPTIONS_PER_MASK = 2 # Note this grows exponentially with number of masks (n). 3^n

In [ ]:
inputs = tokenizer([COMBINED], return_tensors="np")
token_logits = model(**inputs).logits
mask_token_indices = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[:,1]

token_options = {}
for mask_ind in mask_token_indices:
  mask_token_logits = token_logits[0, mask_ind, :]
  top_n_tokens = np.argsort(-mask_token_logits)[:OPTIONS_PER_MASK].tolist()
  token_options[mask_ind] = top_n_tokens

def recursePrintAll(curString, cur_mask_indices, token_options):
  if len(cur_mask_indices) == 0:
    print(curString)
  else:
    for token in token_options[cur_mask_indices[0]]:
      recursePrintAll(curString.replace(tokenizer.mask_token, tokenizer.decode([token]), 1), cur_mask_indices[1:], token_options)

recursePrintAll(STRUCTURE, mask_token_indices, token_options)


Today I went to work but I checked my phone and saw that the stock market collapsed.
Today I went to work but I checked my phone and saw that the stock market crashed.
Today I went to work but I checked my phone and saw that the money market collapsed.
Today I went to work but I checked my phone and saw that the money market crashed.
Today I went to lunch but I checked my phone and saw that the stock market collapsed.
Today I went to lunch but I checked my phone and saw that the stock market crashed.
Today I went to lunch but I checked my phone and saw that the money market collapsed.
Today I went to lunch but I checked my phone and saw that the money market crashed.


In [ ]:
inputs = tokenizer([STRUCTURE], return_tensors="np")
mask_token_indices = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[:,1]

def recurseAssign(structure, topics, cur_mask_indices):
  if '[MASK]' not in structure:
    print(structure)
  else:
    combined = structure + " " + " ".join(topics)
    mask_token_logits = token_logits[0, cur_mask_indices[0], :]
    top_n_tokens = np.argsort(-mask_token_logits)[:OPTIONS_PER_MASK].tolist()
    for token in top_n_tokens:
      recurseAssign(structure.replace(tokenizer.mask_token, tokenizer.decode([token]),1), topics, cur_mask_indices[1:])

recurseAssign(STRUCTURE, TOPICS, mask_token_indices)

Today I went to work but I checked my phone and saw that the stock market collapsed.
Today I went to work but I checked my phone and saw that the stock market crashed.
Today I went to work but I checked my phone and saw that the money market collapsed.
Today I went to work but I checked my phone and saw that the money market crashed.
Today I went to lunch but I checked my phone and saw that the stock market collapsed.
Today I went to lunch but I checked my phone and saw that the stock market crashed.
Today I went to lunch but I checked my phone and saw that the money market collapsed.
Today I went to lunch but I checked my phone and saw that the money market crashed.
